Cleaning des données Cdiscount :
- module de netoyage utf8 vers ascii
- module d'extraction et cleaning des nombres dans les descriptions
- module de lematisation rudimentaire

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import unicodedata as uni
import re
def normalize(string):
    return uni.normalize('NFKD',string).encode('ascii','ignore')
from sys import stdout


Configuration to set

In [2]:
TRAINING_SET_PATH = 'Data/training.csv'
TEST_SET_PATH = 'Data/test.csv'
SEPARATOR = ';'
INDEX_COL = 'Identifiant_Produit'
DESCRIPTION_COL = 'Description'
LIBELLE_COL = 'Libelle'
MARQUE_COL = 'Marque'
MAX_PRICE = 1000
PRICE_COL = 'prix'
AUGMENTED_DESC_COL = 'Description+Libelle'
SAVE_TRAINING_PATH = 'Data/cleantrainV2.csv'
SAVE_TEST_PATH = 'Data/cleantestV2.csv'
STOPLIST = set(('un une le les la au aux de des ce se ma mon ton ' +
               'sa the of then it ou and tres sur on en in dans ' +
               'pour for with avec et ne pas tout all without ' +
               'sans du il votre son one je vous nos ses vos est ' +
               'tu es cet cette tout toute comme comment sommes').split())

Regex to detect pattern in the description:
- page format
- surface
- volume 
- metric
- rapport
- modify livré en expedie pour ne pas confondre avec livre.
- remove numbers

In [3]:
def clean_and_detect_number(line):
    page_info = re.compile('[Aa]+[0-5]+')
    surface_info = re.compile('[\d,\.]+\s?x\s?[\d,\.]+\s?((cm|mm))')
    volume_info = re.compile('l?\s?[\d,\.]+\s?x\s?l?\s?[\d,\.]+\s?x\s?h?\s?[\d,\.]+\s?')
    quantity_info = re.compile('[\d,\.]+\s?x\s?[\d,\.]+\s?((cl|dl|g))')
    metric_info = re.compile('(\d[\d,\.]{0,9})\s?(l|m|w|g|c|v)\s?([^[a-zA-Z]|$|\s])')
    power_rapport = re.compile('\d[\d,\.]{0,9}\s?([a-zA-Z]{1,4})\s?/\s?\d[\d,\.]{0,9}\s?([a-zA-Z]{1,4})')
    percent_info = re.compile('\d+\s?%')
    livre = re.compile('livré|livrés|livre unité|livre unite|livres unités|livres unités')
    regex = re.compile('\d')
    res = regex.findall(line)
    regexs = {'expedie': livre, 'quant':quantity_info,'volume':volume_info,'surface':surface_info,'page':page_info,'metric':metric_info,'percent':percent_info,'rapport':power_rapport}
    if len(res)>0:
        for reg in ['expedie','volume','surface','page','percent','quant','rapport','metric']:
            res2 = regexs[reg].findall(line)
            if len(res2) > 0:
                if reg == 'metric':
                    line = regexs[reg].sub(' ',line)
                    for ine in res2:
                        if ine[1] == 'g':
                            line += ' gr'
                        elif ine[1] == 'v':
                            line += ' volt'
                        elif ine[1] == 'm':
                            line += ' metre'
                        elif ine[1] == 'l':
                            line += ' litre'
                        elif ine[1] == 'w':
                            line += ' watt'      
                else:
                    line = regexs[reg].sub(' '+reg+' ',line)
        line = regex.sub(' ',line)
    return line

In [4]:
regexmarque = re.compile('[^a-zA-Z0-9]')
def detect_marque(line):
    new_line = line
    if line[MARQUE_COL] != 'AUCUNE' and line[MARQUE_COL] != ' ':
        marque_tag = ''.join(regexmarque.sub(" ",str(normalize(line[MARQUE_COL].decode("utf8")))).lower().split() + ['mq'])
        marque_tag = re.compile('\d').sub('n',marque_tag)
        insensitive_marque = re.compile(re.escape(line[MARQUE_COL]), re.IGNORECASE)
        new_line[DESCRIPTION_COL] = insensitive_marque.sub(marque_tag,new_line[DESCRIPTION_COL])
        new_line[LIBELLE_COL] = insensitive_marque.sub(marque_tag,new_line[LIBELLE_COL])
        new_line[MARQUE_COL] = marque_tag
    else:
        new_line[MARQUE_COL] = 'aucunemarque'
    return new_line   

Loading Data through Pandas

In [5]:
print "loading Data"
Ctrain = pd.read_csv(TRAINING_SET_PATH, sep=SEPARATOR,index_col=INDEX_COL)
Ctrain.fillna(' ',inplace=True)
Ctest = pd.read_csv(TEST_SET_PATH, sep=SEPARATOR,index_col=INDEX_COL)
Ctest.fillna(' ',inplace=True)
Ctrain.head(5)

loading Data


,Categorie1,Categorie2,Categorie3,Description,Libelle,Marque,Produit_Cdiscount,prix
Identifiant_Produit,,,,,,,,
0,1000014006,1000015308,1000015309,De Collectif aux éditions SOLESMES,Benedictions de l eglise,,0,3.70
1,1000014006,1000015308,1000015309,De Collectif aux éditions SOLESMES,Notice de st benoit lot de 10,,0,0.95
2,1000014006,1000015308,1000015309,De Richard A M Pe aux éditions TEQUI,Signes pour notre temps fatima,,0,21.85
3,1000014006,1000015308,1000015309,De Mocquereau (Dom Andr aux éditions SOLESMES,Nombre musical t.1,,0,26.60
4,1000014006,1000015308,1000015309,De Collectif aux éditions NS VIDEO,Creation de l homme v522,,0,7.12


Apply regex to remove numbers and special characters

In [6]:
cuts = np.r_[np.arange(0,15000001,1000000),[15786885]]

In [ ]:
print "create Marque repository"
Ctest.ix[:,[DESCRIPTION_COL,LIBELLE_COL,MARQUE_COL]] = Ctest.ix[:,[DESCRIPTION_COL,LIBELLE_COL,MARQUE_COL]].apply(lambda x: detect_marque(x),axis=1)
for i in range(cuts.shape[0]-1):
    Ctrain.ix[cuts[i]:cuts[i+1]-1,[DESCRIPTION_COL,LIBELLE_COL,MARQUE_COL]] = Ctrain.ix[cuts[i]:cuts[i+1]-1,[DESCRIPTION_COL,LIBELLE_COL,MARQUE_COL]].apply(lambda x: detect_marque(x),axis=1)
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()
print " "
print "Done"

create Marque repository
[--                              ] 6.25

In [ ]:
regex = re.compile('[^a-zA-Z]')
regexmarque = re.compile('[^a-zA-Z]')

print "cleaning Libelle"
Ctest[LIBELLE_COL] = Ctest[LIBELLE_COL].astype(str)
Ctest[LIBELLE_COL] = Ctest[LIBELLE_COL].apply(lambda x: clean_and_detect_number(x.decode('utf-8').lower()))
Ctest[LIBELLE_COL] = Ctest[LIBELLE_COL].apply(lambda x: regex.sub(" ",str(normalize(x))))

Ctrain[LIBELLE_COL] = Ctrain[LIBELLE_COL].astype(str)
for i in range(cuts.shape[0]-1):
    Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBELLE_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBELLE_COL].apply(lambda x: clean_and_detect_number(x.decode('utf-8').lower()))
    Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBELLE_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBELLE_COL].apply(lambda x: regex.sub(" ",str(normalize(x))))
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()
print " "
print "Done"

print "cleaning Description"
Ctest[DESCRIPTION_COL] = Ctest[DESCRIPTION_COL].astype(str)
Ctest[DESCRIPTION_COL] = Ctest[DESCRIPTION_COL].apply(lambda x: clean_and_detect_number(x.decode('utf-8').lower()))
Ctest[DESCRIPTION_COL] = Ctest[DESCRIPTION_COL].apply(lambda x: regex.sub(" ",str(normalize(x))))
Ctrain[DESCRIPTION_COL] = Ctrain[DESCRIPTION_COL].astype(str)
for i in range(cuts.shape[0]-1):
    Ctrain.ix[cuts[i]:cuts[i+1]-1,DESCRIPTION_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,DESCRIPTION_COL].apply(lambda x: clean_and_detect_number(x.decode('utf-8').lower()))
    Ctrain.ix[cuts[i]:cuts[i+1]-1,DESCRIPTION_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,DESCRIPTION_COL].apply(lambda x: regex.sub(" ",str(normalize(x))))
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()
print " "
print "Done"

#print "cleaning Marque"
#Ctest[MARQUE_COL] = Ctest[MARQUE_COL].astype(str)
#Ctest[MARQUE_COL] = Ctest[MARQUE_COL].apply(lambda x : ' ' if x == 'AUCUNE' else x) 
#Ctest[MARQUE_COL] = Ctest[MARQUE_COL].apply(lambda x : regexmarque.sub(" ",str(normalize(x.decode("utf8")))).lower())
#Ctrain[MARQUE_COL] = Ctrain[MARQUE_COL].astype(str)
#for i in range(cuts.shape[0]-1):
#    Ctrain.ix[cuts[i]:cuts[i+1]-1,MARQUE_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,MARQUE_COL].apply(lambda x : ' ' if x == 'AUCUNE' else x) 
#    Ctrain.ix[cuts[i]:cuts[i+1]-1,MARQUE_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,MARQUE_COL].apply(lambda x : regexmarque.sub(" ",str(normalize(x.decode("utf8")))).lower())
#    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
#    stdout.flush()
#print " "

In [ ]:
def clean_price(price,max_price):
    clean_price = price
    if price >= max_price and (price%100 ==0 or price*100%10000 == 9999):
        clean_price /= 100
    return clean_price

print "cleaning price"
Ctrain[PRICE_COL] = Ctrain[PRICE_COL].apply(lambda x: clean_price(x,MAX_PRICE))
Ctest[PRICE_COL] = Ctest[PRICE_COL].apply(lambda x: clean_price(x,MAX_PRICE))

Merge Description Libelle and Marque in one big augmented Description

In [ ]:
#Ctrain[AUGMENTED_DESC_COL] = Ctrain[[DESCRIPTION_COL,LIBELLE_COL,MARQUE_COL]].apply(lambda x : x[DESCRIPTION_COL]+" "+x[LIBELLE_COL]+" "+x[MARQUE_COL],axis=1)
#Ctrain.drop([DESCRIPTION_COL,LIBELLE_COL],axis=1,inplace=True)
#Ctest[AUGMENTED_DESC_COL] = Ctest[[DESCRIPTION_COL,LIBELLE_COL,MARQUE_COL]].apply(lambda x : x[DESCRIPTION_COL]+" "+x[LIBELLE_COL]+" "+x[MARQUE_COL],axis=1)
#Ctest.drop([DESCRIPTION_COL,LIBELLE_COL],axis=1,inplace=True)
#Ctrain.head(5)
#Ctest.head(5)

In [ ]:
Ctrain.to_csv(SAVE_TRAINING_PATH)
Ctest.to_csv(SAVE_TEST_PATH)

Completed V1: estimated time of completion 1h00 for 4Go de data in local

Objectif V2: see spark data to implement dataframe RDD.